# Model 1: Baseline

This notebook contains the construction and training of the baseline model. The notebook is split up into four sections: training mode selection (where the model will run), set-up, model constrution, and training. 

Evaluation will take place in the *model_optimization_and_evaluation.ipynb* notebook found in the *notebooks* folder.

In [1]:
# importing necessary packages and libraries
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import optimizers
from keras import applications 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


## Step 1: Training Mode Selection

In the cell below, specify the training mode for the model. This will determine the location from which the source data is drawn, and to which the trained models (and training histories) are saved. 

- **training_mode = 'floydhub'** (runs on Floydhub)
- **training_mode = 'local'** (runs on local disk and processor)

In [2]:
# select training mode
training_mode = 'floydhub'

## Step 2: Set-up

In [3]:
# directory base paths
data_path_local = '../../data/0002_array_data/train_data/'
model_path_local = '../../notebooks/model_construction/saved_models/'
data_path_floydhub = '/floyd/input/capstone_mushrooms/'
model_path_floydhub = '/floyd/home/'

# setting directory paths based on training mode selection
if training_mode == 'floydhub':
    data_path = data_path_floydhub
    model_path = model_path_floydhub
    if training_mode == 'local':
        data_path = data_path_local
        model_path = model_path_local
else:
    raise Exception('Please choose valid training mode: "floydhub" or "local".')

In [4]:
# loading the training and validation data subsets
X_train = np.load(f'{data_path}X_train_data.npy')
y_train = np.load(f'{data_path}y_train_data.npy')
X_val = np.load(f'{data_path}X_val_data.npy')
y_val = np.load(f'{data_path}y_val_data.npy')

In [5]:
# setting training parameters
batch_size = 32
n_classes = 20
n_epochs = 15
img_shape = X_train.shape[1:]
model_names = []
model_list = []
model_hists = []

## Step 3: Model Construction

### Trial 1 - Simple Baseline

In [6]:
# defining the model architecture
model_01 = models.Sequential()

# convolution/max pool stacks
model_01.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=img_shape, padding='same'))
model_01.add(layers.MaxPooling2D((2,2)))

model_01.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
model_01.add(layers.MaxPooling2D((2,2)))

model_01.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
model_01.add(layers.MaxPooling2D((2,2)))

model_01.add(layers.Conv2D(256,(3,3), activation='relu', padding='same'))
model_01.add(layers.MaxPooling2D((2,2)))

# fully connected layers
model_01.add(layers.Flatten())
model_01.add(layers.Dense(512, activation='relu'))
model_01.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
model_01.summary()
model_names.append('model_01')
model_list.append(model_01)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 256)      

## Step 4: Training

In [7]:
# setting up standardization and augmentation parameters
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
%%time
# data standardization and augmentation
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

CPU times: user 387 ms, sys: 945 ms, total: 1.33 s
Wall time: 1.33 s


In [9]:
# compiling loss functions
model_01.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [10]:
%%time
# setting up model saving checkpoints
model_01_cp = ModelCheckpoint(filepath=f'{model_path}model_01.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
model_01_history = model_01.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[model_01_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(model_01_history)

Epoch 1/15
253/253 [==============================] - 97s 383ms/step - loss: 2.6000 - acc: 0.1677 - val_loss: 1.9802 - val_acc: 0.2425

Epoch 00001: val_acc improved from -inf to 0.24250, saving model to /floyd/home/model_01.h5
Epoch 2/15
253/253 [==============================] - 30s 120ms/step - loss: 2.1427 - acc: 0.3093 - val_loss: 1.8360 - val_acc: 0.3535

Epoch 00002: val_acc improved from 0.24250 to 0.35347, saving model to /floyd/home/model_01.h5
Epoch 3/15
253/253 [==============================] - 30s 120ms/step - loss: 1.8194 - acc: 0.4156 - val_loss: 1.5710 - val_acc: 0.4087

Epoch 00003: val_acc improved from 0.35347 to 0.40874, saving model to /floyd/home/model_01.h5
Epoch 4/15
253/253 [==============================] - 31s 121ms/step - loss: 1.5388 - acc: 0.5007 - val_loss: 2.2803 - val_acc: 0.4190

Epoch 00004: val_acc improved from 0.40874 to 0.41902, saving model to /floyd/home/model_01.h5
Epoch 5/15
253/253 [==============================] - 30s 120ms/step - loss: 1.

In [11]:
# creating dictionary for model names, models, and histories from respective lists
models_dict = {i:[j,k] for i,j,k in zip(model_names,model_list,model_hists)}

In [12]:
# evaluating models on validation set
for key, value in models_dict.items():
    (val_loss, val_accuracy) = value[0].evaluate(val_generator,verbose=1)
    print(f'{key} Val Accuracy: {round((val_accuracy*100),2)}%')
    print(f'{key} Val Loss: {round(val_loss,4)}')
    print('---')

26/26 [==============================] - 1s 42ms/step
model_01 Val Accuracy: 42.35%
model_01 Val Loss: 4.7797
---


In [13]:
# saving training histories
for key, value in models_dict.items():
    with open(f'{model_path}{key}_history', 'wb') as file_pi:
        pickle.dump(value[1].history, file_pi)
    print(f'{key}_history saved in {model_path}')

model_01_history saved in /floyd/home/
